In [1]:
cd C:\Users\Alfiya\Desktop\SLAB-main\Code

C:\Users\Alfiya\Desktop\SLAB-main\Code


In [2]:
import os
#Necessary packages
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import os
import joblib 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.models import load_model

from supervised_models import logit, xgb_model, mlp
from utils import perf_metric
from VIME.vime_self import vime_self
from hexr_self import hexr_self

In [3]:
def one_hot_encode(df, class_label):
    #one-hot encode class column
    ohe = OneHotEncoder()
    #Choose either class1 or class2 to select either valence or arousal
    df_ohe = pd.DataFrame(ohe.fit_transform(df[[class_label]]).toarray())
    df = df.join(df_ohe)
    
    df.drop('class', axis=1, inplace=True)
    
    X = df.loc[:,:'gyr_maxabssum']
    y = df.iloc[:,22:]
    return X, y

In [4]:
def split_data_with_preprocessing(X, y, label_data_rate, test_size=0.2):
    """
    Train-test split and train split into unlablled and labelled data with standardization and normalization
    if Test is 15% of the data
    10% of 85% = 8.5% :- Labelled dataset
    90% of 85% = 76.5% :- Unlabelled dataset
    
    if Test is 20% of the data 
    10% of 80% = 8% :- Labelled dataset
    90% of 80% = 72% :- Unlabelled dataset
    """
    from sklearn.preprocessing import StandardScaler, MinMaxScaler

    norm_scaler = MinMaxScaler()

    X[['acc_x_mean', 'acc_x_var', 'acc_y_mean', 'acc_y_var', 'acc_z_mean', 'acc_z_var', 'acc_sum_mean', 'acc_abssum_mean', 'acc_sum_var', 'acc_abssum_var', 'acc_maxabssum','gyr_x_mean', 'gyr_x_var', 'gyr_y_mean', 'gyr_y_var', 'gyr_z_mean', 'gyr_z_var', 'gyr_sum_mean', 'gyr_abssum_mean', 'gyr_sum_var', 'gyr_abssum_var', 'gyr_maxabssum']] = norm_scaler.fit_transform(X[['acc_x_mean', 'acc_x_var', 'acc_y_mean', 'acc_y_var', 'acc_z_mean', 'acc_z_var', 'acc_sum_mean', 'acc_abssum_mean', 'acc_sum_var', 'acc_abssum_var', 'acc_maxabssum','gyr_x_mean', 'gyr_x_var', 'gyr_y_mean', 'gyr_y_var', 'gyr_z_mean', 'gyr_z_var', 'gyr_sum_mean', 'gyr_abssum_mean', 'gyr_sum_var', 'gyr_abssum_var', 'gyr_maxabssum']])
    
    std_scaler = StandardScaler()
    
    X[['acc_x_mean', 'acc_x_var', 'acc_y_mean', 'acc_y_var', 'acc_z_mean', 'acc_z_var', 'acc_sum_mean', 'acc_abssum_mean', 'acc_sum_var', 'acc_abssum_var', 'acc_maxabssum','gyr_x_mean', 'gyr_x_var', 'gyr_y_mean', 'gyr_y_var', 'gyr_z_mean', 'gyr_z_var', 'gyr_sum_mean', 'gyr_abssum_mean', 'gyr_sum_var', 'gyr_abssum_var', 'gyr_maxabssum']] = std_scaler.fit_transform(X[['acc_x_mean', 'acc_x_var', 'acc_y_mean', 'acc_y_var', 'acc_z_mean', 'acc_z_var', 'acc_sum_mean', 'acc_abssum_mean', 'acc_sum_var', 'acc_abssum_var', 'acc_maxabssum','gyr_x_mean', 'gyr_x_var', 'gyr_y_mean', 'gyr_y_var', 'gyr_z_mean', 'gyr_z_var', 'gyr_sum_mean', 'gyr_abssum_mean', 'gyr_sum_var', 'gyr_abssum_var', 'gyr_maxabssum']])

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=33) 
    
    #converting to numpy arrays
    x_train = x_train.iloc[:, :].values
    y_train = y_train.iloc[:, :].values
    x_test = x_test.iloc[:,:].values
    y_test = y_test.iloc[:,:].values
    x_train.shape, x_test.shape, y_train.shape, y_test.shape
    
    # Divide labeled and unlabeled data
    idx = np.random.permutation(len(y_train))
    
    # Label data : Unlabeled data = label_data_rate:(1-label_data_rate)
    label_idx = idx[:int(len(idx)*label_data_rate)]
    unlab_idx = idx[int(len(idx)*label_data_rate):]
    
    # Unlabeled data
    x_unlab = x_train[unlab_idx, :]
    
    # Labeled data
    x_train = x_train[label_idx, :] 
    y_train = y_train[label_idx, :]
    return x_unlab, x_train, x_test, y_train, y_test

In [5]:
#Experimental parameters
label_data_rate = 0.1

#Metrics
metric1 = 'acc'
metric2 = 'auc'

#parameters to train the encoder
p_m = 0.3
alpha = 2.0
K = 3
beta = 1.0

In [6]:
#load case dataset
df = pd.read_excel("..\Data\wheelchair.xlsx")

In [7]:
X, y = one_hot_encode(df, 'class')

In [8]:
x_unlab, x_train, x_test, y_train, y_test = split_data_with_preprocessing(X, y, label_data_rate, test_size=0.15)

In [9]:
# Train SLAB-Self encoder
slab_self_parameters = dict()
slab_self_parameters['batch_size'] = 128
slab_self_parameters['epochs'] = 10
slab_valence_encoder = hexr_self(x_unlab, p_m, alpha, slab_self_parameters)
    
# Save encoder
if not os.path.exists('Models'):
  os.makedirs('Models')

file_name = 'Models/wheel_80_5.h5'
    
slab_valence_encoder.save(file_name)  

Epoch 1/10
166/166 [==============================] - 1s 653us/step - loss: 1.0839 - mask_loss: 0.6408 - feature_loss: 0.2215
Epoch 2/10
166/166 [==============================] - 0s 612us/step - loss: 0.9154 - mask_loss: 0.6165 - feature_loss: 0.1495
Epoch 3/10
166/166 [==============================] - 0s 605us/step - loss: 0.8756 - mask_loss: 0.6157 - feature_loss: 0.1300
Epoch 4/10
166/166 [==============================] - 0s 625us/step - loss: 0.8594 - mask_loss: 0.6149 - feature_loss: 0.1223
Epoch 5/10
166/166 [==============================] - 0s 617us/step - loss: 0.8504 - mask_loss: 0.6143 - feature_loss: 0.1181
Epoch 6/10
166/166 [==============================] - 0s 609us/step - loss: 0.8441 - mask_loss: 0.6138 - feature_loss: 0.1151
Epoch 7/10
166/166 [==============================] - 0s 614us/step - loss: 0.8391 - mask_loss: 0.6134 - feature_loss: 0.1128
Epoch 8/10
166/166 [==============================] - 0s 614us/step - loss: 0.8352 - mask_loss: 0.6131 - feature_loss:

In [10]:
#predict and evaluate slab self encoder
slab_valence_encoder = load_model('Models/wheel_80_5.h5')

# Test SLAB
x_train_hat = slab_valence_encoder.predict(x_train)
x_test_hat = slab_valence_encoder.predict(x_test)

130/130 [==============================] - 0s 419us/step


In [11]:
from utils import convert_vector_to_matrix
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import HeNormal

def mlp(x_train, y_train, x_test, parameters):
  """Multi-layer perceptron (MLP).

  Args:
    - x_train, y_train: training dataset
    - x_test: testing feature
    - parameters: hidden_dim, epochs, activation, batch_size, num_layers

  Returns:
    - y_test_hat: predicted values for x_test
  """

  # Convert labels into proper format
  if len(y_train.shape) == 1:
    y_train = convert_vector_to_matrix(y_train)

  # Divide training and validation sets (9:1)
  idx = np.random.permutation(len(x_train[:, 0]))
  train_idx = idx[:int(len(idx)*0.9)]
  valid_idx = idx[int(len(idx)*0.9):]

  # Validation set
  x_valid = x_train[valid_idx, :]
  y_valid = y_train[valid_idx, :]

  # Training set
  x_train = x_train[train_idx, :]
  y_train = y_train[train_idx, :]

  # Reset the graph
  K.clear_session()

  # Define network parameters
  hidden_dim = parameters['hidden_dim']
  epochs_size = parameters['epochs']
  act_fn = parameters['activation']
  batch_size = parameters['batch_size']
  #std_value = parameters['std_value']

  # Define basic parameters
  data_dim = len(x_train[0, :])
  label_dim = len(y_train[0, :])

  # Build model - with number of layers specified by parameters
  model = Sequential()
  initializer = HeNormal()
  
  model.add(Dense(hidden_dim, input_dim = data_dim, activation = act_fn, kernel_initializer=initializer))
  for i in range(0, parameters['num_layers']):
    model.add(Dense(hidden_dim, activation = act_fn, kernel_initializer=initializer))
  model.add(Dense(label_dim, activation = 'softmax'))

  model.compile(loss = 'categorical_crossentropy', optimizer='adam',
                metrics = ['acc'])

  es = EarlyStopping(monitor='val_loss', mode = 'min',
                     verbose = 1, restore_best_weights=True, patience=50)

  # Fit model on training dataset
  model.fit(x_train, y_train, validation_data = (x_valid, y_valid),
            epochs = epochs_size, batch_size = batch_size,
            verbose = 0, callbacks=[es])

  # Predict on x_test
  y_test_hat = model.predict(x_test)
  #print("Random Normal initialization")
  #print("SLAB MLP was trained on the test.")
  return y_test_hat

In [12]:
#experiment to compute accuracy for different number of hidden layers
wc_acc = []
wc_auc = []
f1_score_weg = []

mlp_parameters = dict()
mlp_parameters['hidden_dim'] = 100
mlp_parameters['epochs'] = 100
mlp_parameters['activation'] = 'relu'
mlp_parameters['batch_size'] = 128
mlp_parameters['num_layers'] = 5

In [ ]:
for i in range (0, 5):
        y_test_hat = mlp(x_train_hat, y_train, x_test_hat, mlp_parameters)
        wc_acc.append(perf_metric(metric1, y_test, y_test_hat))
        wc_auc.append(perf_metric(metric2, y_test, y_test_hat)) 
        f1_score_weg.append(f1_score(y_test.argmax(1),y_test_hat.argmax(1),average="weighted"))

In [22]:
print("Mean of valence accuracies for SLAB is ",round(np.mean(wc_acc),4))

# Compute standard deviation
std_dev_acc = np.std(wc_acc)
print("Standard Deviation of Valence Accuracies using SLAB: ", round(std_dev_acc,4))
    
print("Mean of valence accuracies AUC Score for SLAB is ",round(np.mean(wc_auc),4))

#print("Mean of f1-score mic for SLAB is ",round(np.mean(top_score_mic),4))
#print("Mean of f1-score mac for SLAB is ",round(np.mean(top_score_mac),4))
print("Mean of f1-score weighted for SLAB is ",round(np.mean(f1_score_weg),4))

Mean of valence accuracies for SLAB is  0.707
Standard Deviation of Valence Accuracies using SLAB:  0.0059
Mean of valence accuracies AUC Score for SLAB is  0.9475
Mean of f1-score weighted for SLAB is  0.704
